In [1]:
import tensorflow as tf
import string
import requests

In [81]:
 response=requests.get('put link')

In [ ]:
response

In [ ]:
response.text

In [ ]:
data=response.text.split('\n')
data[1]

In [ ]:
data=data[253:]
data

In [86]:
data=" ".join(data)

In [ ]:
data[200:]

In [ ]:
len(data)

In [89]:
def clean_text(doc):
 tokens=doc.split()
 table=str.maketrans('','',string.punctuation)
 tokens=[w.translate(table) for w in tokens]
 tokens=[word for word in  tokens if word.isalpha()]
 tokens=[word.lower for word in tokens]
 return tokens

In [ ]:
tokens=clean_text(data)
print(tokens[300:])

In [ ]:
len(tokens)

In [ ]:
len(set(tokens))

In [ ]:
length=50 + 1
lines=[]

for i in range(length,len(tokens)):
  seq=tokens[i-length:i]
  line= ','.join(str(v) for v in seq)
  lines.append(line)
  if i > 200:
    break

print(len(lines))

In [ ]:
lines[1]

In [93]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [94]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(lines)
sequences=tokenizer.texts_to_sequences(lines)

In [95]:
sequences=np.array(sequences)
X,y=sequences[:, :-1],sequences[:,-1]

In [ ]:
X[0]

In [ ]:
y[0]

In [97]:
vocab_size=len(tokenizer.word_index) + 1

In [ ]:
y=to_categorical(y,num_classes=vocab_size)
X.shape[1]

In [99]:
seq_length=X.shape[1]

In [100]:
model=Sequential()
model.add(Embedding(vocab_size,50,input_length=seq_length))
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100,activation='relu'))
model.add(Dense(vocab_size,activation='softmax'))

In [ ]:
model.summary()

In [102]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(X,y,batch_size=5, epochs = 30)

In [115]:
def generate_text_seq(model,tokenizer,text_seq_length,seed_text,n_words):
  text=[]
  for _ in range(n_words):
    encoded=tokenizer.texts_to_sequences([seed_text])[0]
    encoded=pad_sequences([encoded],maxlen=text_seq_length,truncating='pre')
    y_predict=model.predict_classes(encoded)
    predicted_word=''
    for word,index in tokenizer.word_index.items():
      if index==y_predict:
        predicted_word=word
        break

        seed_text=seed_text + ' '+ predicted_word
        text.append(predicted_word)
        return ' '.join(text)

In [ ]:
generate_text_seq(model,tokenizer,seq_length,seed_text,100)

In [ ]:
seed_text